# **Importing Necessary Libraries**

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from tensorflow.python.keras import Sequential
from tensorflow.python.keras import layers
from tensorflow.python.keras import regularizers

from google.colab import drive
import os
import glob
drive.mount('/content/drive/')
os.chdir('/content/drive/My Drive/CA684_Assignment/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# **Definations**

In [0]:
def read_C3D(fname):
    """Scan vectors from file"""
    with open(fname) as f:
        for line in f:
            C3D =[float(item) for item in line.split()] # convert to float type, using default separator
    return C3D

In [0]:
def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    lock = True
    #x = 0
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            #print(score_mat)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

### **Getting Ground Truth file**

In [16]:
ground_truth_file = '/content/drive/My Drive/CA684_Assignment/Dev-set/Ground-truth/ground-truth.csv'
ground_truth = pd.read_csv(ground_truth_file)
ground_truth.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3.webm,0.924,34,0.846,13
1,video4.webm,0.923,33,0.667,12
2,video6.webm,0.863,33,0.700,10
3,video8.webm,0.922,33,0.818,11
4,video10.webm,0.950,34,0.900,10


#**Captions**

### **Getting Captions**

In [17]:
# load labels and captions
def read_caps(fname):
    """Load the captions into a dataframe"""
    vn = []
    cap = []
    df = pd.DataFrame();
    with open(fname) as f:
        for line in f:
            pairs = line.split()
            vn.append(pairs[0])
            cap.append(pairs[1])
        df['video']=vn
        df['caption']=cap
    return df


# load the captions
cap_path = '/content/drive/My Drive/CA684_Assignment/Dev-set/Captions/dev-set_video-captions.txt'
df_cap=read_caps(cap_path)
df_cap.head()

,video,caption
0,video3.webm,blonde-woman-is-massaged-tilt-down
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot
2,video6.webm,khr-gangsters
3,video8.webm,medical-helicopter-hovers-at-airport
4,video10.webm,couple-relaxing-on-picnic-crane-shot


### **Removing Stopwords from captions**

In [18]:
!pip install PyPrind
from string import punctuation
import pyprind
from collections import Counter
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
#stopwords = nltk.corpus.stopwords.words('english')
stopwords = stopwords.words('english')
print(f'Length of Stopwords: {len(stopwords)}')

# setup prograss tracker
pbar = pyprind.ProgBar(len(df_cap['caption']), title='Counting word occurrences')
for i, cap in enumerate(df_cap['caption']):
    # replace punctuations with space
    # convert words to lower case 
    text = ''.join([c if c not in punctuation else ' ' for c in cap]).lower()
    #removing stopwords
    rmv_stopwords= ' '.join([word for word in text.split() if word not in stopwords])
    df_cap.loc[i,'caption'] = rmv_stopwords #updating the original captions 
    pbar.update()

print("")
print(df_cap.head())
print(df_cap.shape)

Counting word occurrences


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Length of Stopwords: 179


0% [##############################] 100% | ETA: 00:00:00


          video                                    caption
0   video3.webm                 blonde woman massaged tilt
1   video4.webm  roulette table spinning ball closeup shot
2   video6.webm                              khr gangsters
3   video8.webm          medical helicopter hovers airport
4  video10.webm          couple relaxing picnic crane shot
(6000, 2)



Total time elapsed: 00:00:01


### **Implementing models on caption individually**

**Dividing Dataset into train and test**

In [21]:
y = ground_truth[['short-term_memorability','long-term_memorability']].values # targets
X = X_arrseq
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42) # random state for reproducability
print(X.shape)
print(y.shape)

print('X_train ', X_train.shape)
print('X_test  ', X_test.shape)
print('Y_train ', y_train.shape)
print('Y_test  ', y_test.shape)

(6000, 5087)
(6000, 2)
X_train  (4800, 5087)
X_test   (1200, 5087)
Y_train  (4800, 2)
Y_test   (1200, 2)


**SVR model on Captions**

In [22]:
svr = SVR()
model = MultiOutputRegressor(estimator=svr)
model.fit(X_train,y_train)
predictions_cap_svr = model.predict(X_test)
Get_score(predictions_cap_svr, y_test)

The Spearman's correlation coefficient is: 0.421
The Spearman's correlation coefficient is: 0.187


**Random Forest on Captions**

In [23]:
rf = RandomForestRegressor()
model = MultiOutputRegressor(estimator=rf)
model.fit(X_train,y_train)
print(model.score(X_test,y_test))
predictions_cap_rf = model.predict(X_test)
Get_score(predictions_cap_rf, y_test)

0.041584359761635714
The Spearman's correlation coefficient is: 0.423
The Spearman's correlation coefficient is: 0.186


**LASSO on Captions**

In [24]:
from sklearn.linear_model import Lasso
ls = Lasso(alpha=1e-5,normalize=True, max_iter=1e5)
model = MultiOutputRegressor(estimator=ls)
model.fit(X_train,y_train)
print(model.score(X_test,y_test))
predictions_cap_lasso = model.predict(X_test)
Get_score(predictions_cap_lasso, y_test)

-0.04043569421669718
The Spearman's correlation coefficient is: 0.417
The Spearman's correlation coefficient is: 0.163


**Taking average of SVR and LASSO for Captions**

In [25]:
predictions_weighted_cap  = (1/2)*predictions_cap_svr + (1/2)*predictions_cap_lasso
print(predictions_weighted_cap)
Get_score(predictions_weighted_cap, y_test)

[[0.88670645 0.77384876]
 [0.87507066 0.78874734]
 [0.88053736 0.85425237]
 ...
 [0.88585137 0.87559943]
 [0.86346641 0.78248466]
 [0.84789084 0.72888865]]
The Spearman's correlation coefficient is: 0.454
The Spearman's correlation coefficient is: 0.195


**Taking average of SVR and RF for Captions**

In [27]:
predictions_weighted_cap1  = (1/2)*predictions_cap_svr + (1/2)*predictions_cap_rf
print(predictions_weighted_cap1)
Get_score(predictions_weighted_cap1, y_test)

[[0.84765621 0.77493978]
 [0.83474412 0.77312901]
 [0.86508299 0.82831865]
 ...
 [0.85941627 0.84926681]
 [0.87642799 0.79505453]
 [0.8482377  0.77987116]]
The Spearman's correlation coefficient is: 0.454
The Spearman's correlation coefficient is: 0.197


**Taking average of LASSO and RF for Captions**

In [28]:
predictions_weighted_cap2  = (1/2)*predictions_cap_lasso + (1/2)*predictions_cap_rf
print(predictions_weighted_cap2)
Get_score(predictions_weighted_cap2, y_test)

[[0.88114514 0.78716231]
 [0.84846321 0.75499497]
 [0.89256998 0.85149015]
 ...
 [0.8933554  0.90626728]
 [0.88046275 0.7858693 ]
 [0.86600345 0.76124749]]
The Spearman's correlation coefficient is: 0.449
The Spearman's correlation coefficient is: 0.191


**Taking average of SVR,LASSO and RF for Captions**

In [29]:
predictions_weighted_cap3  = (1/3)*predictions_cap_svr + (1/3)*predictions_cap_rf + (1/3)*predictions_cap_lasso
print(predictions_weighted_cap3)
Get_score(predictions_weighted_cap3, y_test)

[[0.87183593 0.77865028]
 [0.85275933 0.77229044]
 [0.87939678 0.84468706]
 ...
 [0.87954102 0.87704451]
 [0.87345238 0.78780283]
 [0.854044   0.7566691 ]]
The Spearman's correlation coefficient is: 0.465
The Spearman's correlation coefficient is: 0.201


#**C3D**

### **Getting C3D Features and preprocessing**

In [0]:
os.chdir('/content/drive/My Drive/CA684_Assignment/Dev-set/')

In [33]:
C3D_data = []
entries = os.listdir('/content/drive/My Drive/CA684_Assignment/Dev-set/C3D/')

for entry in entries:
  #print(entry)
  example_inception_path = '/content/drive/My Drive/CA684_Assignment/Dev-set/C3D/'+entry
  txt_data = pd.read_csv(example_inception_path, sep=" ", header=None)
  C3D_data.append(txt_data)
C3D_features = pd.concat(C3D_data)
C3D_features = C3D_features.drop(columns=[101])
C3D_features.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,0.010858,0.010386,0.000000,0.000000e+00,0.000000e+00,0.000000,2.700000e-07,0.000000,1.000000e-08,3.400000e-07,8.000000e-08,1.000000e-08,0.000004,0.000105,0.000000e+00,4.000000e-08,1.000000e-08,0.000000,0.000000,1.331800e-04,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,7.600000e-07,0.000000,9.800000e-07,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,6.000000e-08,0.000021,5.000000e-08,0.000000e+00,1.000000e-08,3.000000e-08,2.000000e-08,0.000000e+00,...,0.001623,0.970125,0.000016,0.001298,0.000032,0.000001,0.000000,0.000000,1.000000e-08,2.500000e-07,0.000000e+00,0.000000,5.000000e-08,0.000000,1.000000e-08,1.000000e-08,4.200000e-07,0.000000e+00,3.000000e-08,2.000000e-08,0.000000,0.000000,0.000000,6.000000e-08,0.000000,0.000000,9.000000e-08,0.000000,0.000000e+00,1.100000e-07,1.400000e-07,0.000000,0.000000,1.700000e-07,0.000000,0.000000e+00,1.000000e-08,1.300000e-06,2.600000e-06,8.000000e-08
0,0.000200,0.000065,0.993807,2.000000e-07,4.700000e-07,0.000073,3.700000e-06,0.000337,6.710000e-06,2.290000e-06,6.380000e-06,7.340000e-06,0.000019,0.000007,3.240000e-06,3.810000e-06,1.411000e-05,0.000001,0.000003,2.300000e-07,1.900000e-07,0.002938,5.200000e-07,9.200000e-07,1.136000e-05,0.000011,3.310400e-04,9.200000e-07,0.000022,8.000000e-08,0.000028,0.000013,5.778000e-05,0.000037,1.683000e-05,6.860000e-06,3.990000e-06,8.030000e-06,1.450000e-06,3.800000e-07,...,0.000045,0.000027,0.000013,0.000077,0.000050,0.000138,0.000012,0.000007,4.100000e-07,5.440000e-06,3.400000e-07,0.000007,8.399000e-05,0.000004,3.970000e-06,5.900000e-07,1.285300e-04,8.800000e-07,2.228000e-05,1.105000e-05,0.000003,0.000015,0.000014,4.090000e-06,0.000003,0.000005,4.920000e-06,0.000005,6.400000e-07,2.372000e-05,6.610000e-06,0.000010,0.000002,5.800000e-06,0.000001,1.490000e-06,1.170000e-05,1.500000e-07,8.300000e-07,1.060000e-04
0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000008,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.999985,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.120000e-06
0,0.005959,0.004765,0.003757,5.787100e-04,7.344000e-05,0.000129,4.665300e-04,0.010266,3.028100e-04,2.196500e-04,2.521800e-04,2.963730e-03,0.000032,0.000592,3.260000e-05,1.466430e-03,2.430700e-04,0.000181,0.004019,2.390960e-03,1.206900e-04,0.008488,5.216200e-04,4.164600e-04,1.643420e-02,0.049686,1.470550e-03,5.432000e-05,0.001055,5.442770e-03,0.000173,0.024688,9.640700e-04,0.000880,1.140529e-02,4.522160e-03,8.594500e-04,6.573000e-04,6.942400e-04,1.823110e-03,...,0.021621,0.000639,0.014109,0.000472,0.002461,0.004046,0.000905,0.000997,2.514660e-03,1.587440e-03,1.125030e-03,0.001008,2.133877e-02,0.000647,1.373010e-03,9.568000e-05,5.635083e-02,1.628000e-05,5.379910e-03,6.020020e-03,0.046499,0.023942,0.020658,3.349520e-03,0.002637,0.000094,1.170705e-02,0.000229,9.789210e-03,2.764390e-03,8.052350e-03,0.003992,0.004137,3.670390e-02,0.000767,3.692100e-04,1.251980e-02,8.422000e-05,1.159825e-02,1.155040e-03
0,0.005782,0.000306,0.004011,1.007000e-05,1.034000e-05,0.000002,3.160000e-06,0.000003,1.984000e-05,5.750000e-06,6.642000e-05

In [36]:
C3D_feature = C3D_features.values
print(C3D_feature)
print(C3D_feature.shape)

[[1.0857670e-02 1.0386330e-02 0.0000000e+00 ... 1.3000000e-06
  2.6000000e-06 8.0000000e-08]
 [1.9951000e-04 6.4940000e-05 9.9380690e-01 ... 1.5000000e-07
  8.3000000e-07 1.0600000e-04]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 7.1200000e-06]
 ...
 [3.0822000e-04 2.0097560e-02 2.0226000e-04 ... 1.0275000e-04
  4.0909850e-02 4.4673800e-03]
 [1.0868380e-02 9.4189000e-04 6.6393300e-03 ... 2.6109700e-02
  1.9424398e-01 6.3640000e-05]
 [1.6464800e-03 7.2505000e-04 1.3462000e-04 ... 6.5269600e-03
  5.7053000e-04 4.9473000e-03]]
(6000, 101)


**Splitting C3D data into test and train**

In [37]:
y = ground_truth[['short-term_memorability','long-term_memorability']].values # targets
X = C3D_feature
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42) # random state for reproducability
print(len(X_train[0]))

101


### **Implementing models on C3D individually**

**Forest on C3D Features**

In [38]:
rf = RandomForestRegressor(n_estimators=200)
model = MultiOutputRegressor(estimator=rf)
model.fit(X_train,y_train)
#print(model.score(X_test,y_test))
predictions_c3d_rf = model.predict(X_test)
Get_score(predictions_c3d_rf, y_test)

The Spearman's correlation coefficient is: 0.015
The Spearman's correlation coefficient is: 0.033


**SVR on C3D Features**

In [39]:
svr = SVR(C=0.0005, epsilon=0.2)
model = MultiOutputRegressor(estimator=svr)
model.fit(X_train,y_train)
#print(model.score(X_test,y_test))
predictions_c3d_svr = model.predict(X_test)
Get_score(predictions_c3d_svr, y_test)

The Spearman's correlation coefficient is: 0.038
The Spearman's correlation coefficient is: 0.019


**LASSO on C3D Features**

In [40]:
from sklearn.linear_model import Lasso
ls = Lasso(alpha=1e-5,normalize=True, max_iter=1e5)
model = MultiOutputRegressor(estimator=ls)
model.fit(X_train,y_train)
print(model.score(X_test,y_test))
predictions_c3d_lasso = model.predict(X_test)
Get_score(predictions_c3d_lasso, y_test)

-0.017883403648563223
The Spearman's correlation coefficient is: 0.015
The Spearman's correlation coefficient is: -0.004


**Taking average of LASSO and SVR and RF for C3D**


In [41]:
predictions_weighted_c3d  = (1/3)*predictions_c3d_lasso + (1/3)*predictions_c3d_svr + (1/3)*predictions_c3d_rf
print(predictions_weighted_c3d)
Get_score(predictions_weighted_c3d, y_test)

[[0.84140513 0.78397052]
 [0.82982892 0.78274636]
 [0.8263758  0.77046888]
 ...
 [0.83779961 0.78678211]
 [0.83576975 0.77934957]
 [0.83885634 0.78166396]]
The Spearman's correlation coefficient is: 0.009
The Spearman's correlation coefficient is: 0.031


**Taking average of LASSO and SVR for C3D**

In [42]:
predictions_weighted_c3d  = (1/2)*predictions_c3d_lasso + (1/2)*predictions_c3d_svr
print(predictions_weighted_c3d)
Get_score(predictions_weighted_c3d, y_test)

[[0.82316519 0.77472577]
 [0.82015089 0.78453455]
 [0.8200137  0.77070081]
 ...
 [0.82306942 0.78436066]
 [0.82260713 0.77586685]
 [0.82261202 0.77906594]]
The Spearman's correlation coefficient is: 0.019
The Spearman's correlation coefficient is: -0.000


**Taking average of RF and SVR for C3D**

In [43]:
predictions_weighted_c3d  = (1/2)*predictions_c3d_rf + (1/2)*predictions_c3d_svr
print(predictions_weighted_c3d)
Get_score(predictions_weighted_c3d, y_test)

[[0.83092139 0.78640902]
 [0.8169522  0.77573796]
 [0.81163661 0.76939746]
 ...
 [0.82595373 0.78207131]
 [0.82322971 0.77910384]
 [0.82775289 0.7799393 ]]
The Spearman's correlation coefficient is: 0.015
The Spearman's correlation coefficient is: 0.034


**Taking average of LASSO and RF for C3D**

In [44]:
predictions_weighted_c3d  = (1/2)*predictions_c3d_lasso + (1/2)*predictions_c3d_rf
print(predictions_weighted_c3d)
Get_score(predictions_weighted_c3d, y_test)

[[0.87012879 0.79077676]
 [0.85238368 0.78796659]
 [0.84747709 0.77130835]
 ...
 [0.86437569 0.79391435]
 [0.86147242 0.78307801]
 [0.86620412 0.78598664]]
The Spearman's correlation coefficient is: 0.008
The Spearman's correlation coefficient is: 0.031


# **Captions + C3D**

### **Concatination and preprocessing**

**Captions and C3D concatination**

In [45]:
Caption_C3D = np.concatenate((X_arrseq, C3D_features), axis=1)
print(Caption_C3D)
print(Caption_C3D.shape)

[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.3000000e-06
  2.6000000e-06 8.0000000e-08]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.5000000e-07
  8.3000000e-07 1.0600000e-04]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 7.1200000e-06]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.0275000e-04
  4.0909850e-02 4.4673800e-03]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 2.6109700e-02
  1.9424398e-01 6.3640000e-05]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 6.5269600e-03
  5.7053000e-04 4.9473000e-03]]
(6000, 5188)


**Dividing Captions and C3D into test and train**

In [46]:
y = ground_truth[['short-term_memorability','long-term_memorability']].values # targets
X = Caption_C3D
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42) # random state for reproducability
print(X.shape)
print(y.shape)

print('X_train ', X_train.shape)
print('X_test  ', X_test.shape)
print('Y_train ', y_train.shape)
print('Y_test  ', y_test.shape)

(6000, 5188)
(6000, 2)
X_train  (4800, 5188)
X_test   (1200, 5188)
Y_train  (4800, 2)
Y_test   (1200, 2)


### **Implementing models on C3D and captions individually**

**Random Forest on Captions and C3D**

In [50]:
rf = RandomForestRegressor(n_estimators = 100)
model = MultiOutputRegressor(estimator=rf)
model.fit(X_train,y_train)
#print(model.score(X_test,y_test))
predictions_cap_c3d_rf = model.predict(X_test)
Get_score(predictions_cap_c3d_rf, y_test)

The Spearman's correlation coefficient is: 0.334
The Spearman's correlation coefficient is: 0.062


**SVR on Captions and C3D**

In [47]:
svr = SVR()
model = MultiOutputRegressor(estimator=svr)
model.fit(X_train,y_train)
#print(model.score(X_test,y_test))
predictions_cap_c3d_svr = model.predict(X_test)
Get_score(predictions_cap_c3d_svr, y_test)

The Spearman's correlation coefficient is: 0.402
The Spearman's correlation coefficient is: 0.198


**LASSO on Captions and C3D**

In [48]:
from sklearn.linear_model import Lasso
ls = Lasso(alpha=1e-5,normalize=True, max_iter=1e5)
model = MultiOutputRegressor(estimator=ls)
model.fit(X_train,y_train)
print(model.score(X_test,y_test))
predictions_cap_c3d_lasso = model.predict(X_test)
Get_score(predictions_cap_c3d_lasso, y_test)

-0.07973299542696144
The Spearman's correlation coefficient is: 0.399
The Spearman's correlation coefficient is: 0.156


**Ensambling results of LASSO and SVR and RF for Captions and C3D**

In [51]:
predictions_weighted_cap_c3d_2  = (1/3)*predictions_cap_c3d_svr + (1/3)*predictions_cap_c3d_lasso + (1/3)*predictions_cap_c3d_rf
print(predictions_weighted_cap_c3d_2)
Get_score(predictions_weighted_cap_c3d_2, y_test)

[[0.86194844 0.78942889]
 [0.86034558 0.78532411]
 [0.86776621 0.81361511]
 ...
 [0.87891308 0.87062317]
 [0.86644828 0.77682304]
 [0.85684908 0.74187301]]
The Spearman's correlation coefficient is: 0.431
The Spearman's correlation coefficient is: 0.182


**Ensambling results of LASSO and SVR for Captions and C3D**

In [49]:
predictions_weighted_cap_c3d_2  = (1/2)*predictions_cap_c3d_svr + (1/2)*predictions_cap_c3d_lasso
print(predictions_weighted_cap_c3d_2)
Get_score(predictions_weighted_cap_c3d_2, y_test)

[[0.87901766 0.78790833]
 [0.86855336 0.80155116]
 [0.87629932 0.84431267]
 ...
 [0.88536961 0.90123976]
 [0.86289242 0.78248455]
 [0.85678862 0.73274951]]
The Spearman's correlation coefficient is: 0.429
The Spearman's correlation coefficient is: 0.188


**Ensambling results of LASSO and RF for Captions and C3D**

In [52]:
predictions_weighted_cap_c3d_2  = (1/2)*predictions_cap_c3d_rf + (1/2)*predictions_cap_c3d_lasso
print(predictions_weighted_cap_c3d_2)
Get_score(predictions_weighted_cap_c3d_2, y_test)

[[0.87301592 0.78893583]
 [0.86501788 0.77204302]
 [0.87910082 0.80829101]
 ...
 [0.89141078 0.89236402]
 [0.87096892 0.76891627]
 [0.86182569 0.73508943]]
The Spearman's correlation coefficient is: 0.412
The Spearman's correlation coefficient is: 0.151


**Ensambling results of RF and SVR for Captions and C3D**

In [53]:
predictions_weighted_cap_c3d_2  = (1/2)*predictions_cap_c3d_rf + (1/2)*predictions_cap_c3d_svr
print(predictions_weighted_cap_c3d_2)
Get_score(predictions_weighted_cap_c3d_2, y_test)

[[0.83381173 0.79144249]
 [0.84746548 0.78237814]
 [0.8478985  0.78824166]
 ...
 [0.85995883 0.81826575]
 [0.8654835  0.77906828]
 [0.85193292 0.75778008]]
The Spearman's correlation coefficient is: 0.403
The Spearman's correlation coefficient is: 0.178


# **Final Result - Test Dataset**


### **Getting C3D Test Data and Preprocessing**

**Getting Testing Dataset of C3D Features**


In [0]:
os.chdir('/content/drive/My Drive/CA684_Assignment/Test-set/')

In [0]:
C3D_feature_test_list= []
video_names_test_list = []
for filename in glob.glob('C3D_test/*.txt'):
    name = ((filename.split('/')[-1]).split('.')[0]+'.webm')
    video_names_test_list.append(name)
    C3D_features_test = read_C3D(filename)
    C3D_feature_test_list.append(C3D_features_test)

C3D_features_test = pd.DataFrame(np.array(C3D_feature_test_list).reshape(2000,101))


In [59]:
C3D_test_features = C3D_features_test.values
print(C3D_test_features)
print(C3D_test_features.shape)

[[3.5262400e-03 1.3763600e-03 4.6187050e-02 ... 5.0880000e-05
  6.6530000e-05 6.8281257e-01]
 [7.7715900e-03 2.3021500e-03 3.6714600e-03 ... 1.7517360e-02
  6.6693252e-01 2.8582300e-03]
 [1.5729500e-03 1.3980800e-03 7.1722990e-02 ... 3.3770000e-05
  8.3744000e-04 8.0448484e-01]
 ...
 [2.1110000e-04 5.8155900e-03 2.3340000e-04 ... 3.2178000e-04
  6.9276000e-04 1.3163320e-02]
 [2.1449800e-03 3.5880870e-02 2.3955000e-04 ... 1.4710000e-05
  1.1600000e-06 7.4280000e-04]
 [1.0272900e-03 8.8060000e-05 1.0535000e-04 ... 6.9846000e-04
  4.9600000e-06 4.1440000e-05]]
(2000, 101)


**Merging C3D Developement and Test Datasets**

In [60]:
C3D_dataset = pd.concat([C3D_features, C3D_features_test], ignore_index=True)
print(C3D_dataset.head())
print(C3D_dataset.tail())

        0         1         2    ...           98            99            100
0  0.010858  0.010386  0.000000  ...  1.300000e-06  2.600000e-06  8.000000e-08
1  0.000200  0.000065  0.993807  ...  1.500000e-07  8.300000e-07  1.060000e-04
2  0.000000  0.000000  0.000000  ...  0.000000e+00  0.000000e+00  7.120000e-06
3  0.005959  0.004765  0.003757  ...  8.422000e-05  1.159825e-02  1.155040e-03
4  0.005782  0.000306  0.004011  ...  1.271750e-03  4.862200e-04  1.965000e-05

[5 rows x 101 columns]
           0         1         2    ...       98        99        100
7995  0.000004  0.000013  0.001774  ...  0.000002  0.000035  0.001458
7996  0.000250  0.000228  0.000109  ...  0.000040  0.000471  0.002141
7997  0.000211  0.005816  0.000233  ...  0.000322  0.000693  0.013163
7998  0.002145  0.035881  0.000240  ...  0.000015  0.000001  0.000743
7999  0.001027  0.000088  0.000105  ...  0.000698  0.000005  0.000041

[5 rows x 101 columns]


### **Getting Captions Data and Preprocessing**

**Getting Training and Testing Dataset of caption**

In [61]:
df_cap.head()

,video,caption
0,video3.webm,blonde woman massaged tilt
1,video4.webm,roulette table spinning ball closeup shot
2,video6.webm,khr gangsters
3,video8.webm,medical helicopter hovers airport
4,video10.webm,couple relaxing picnic crane shot


In [62]:
# load labels and captions
def read_caps(fname):
    """Load the captions into a dataframe"""
    vn = []
    cap = []
    df = pd.DataFrame();
    with open(fname) as f:
        for line in f:
            pairs = line.split()
            vn.append(pairs[0])
            cap.append(pairs[1])
        df['video']=vn
        df['caption']=cap
    return df


# load the captions
cap_test_path = './Captions_test/test-set-1_video-captions.txt'
df_cap_test=read_caps(cap_test_path)
print(df_cap_test.tail())

                video                                            caption
1995  video10004.webm  astronaut-in-outer-space-against-the-backdrop-...
1996  video10005.webm  young-women-lying-on-sunbed-and-applying-sun-c...
1997  video10006.webm  doctor-talking-to-patient-using-a-tablet-to-ex...
1998  video10007.webm  businessman-sitting-on-the-beach-on-inflatable...
1999  video10008.webm  woman-eating-ice-cream-and-sitting-in-the-stre...


In [63]:
Caption_dataset = pd.concat([df_cap, df_cap_test],ignore_index=True)
print(Caption_dataset.head())
print(Caption_dataset.tail())

          video                                    caption
0   video3.webm                 blonde woman massaged tilt
1   video4.webm  roulette table spinning ball closeup shot
2   video6.webm                              khr gangsters
3   video8.webm          medical helicopter hovers airport
4  video10.webm          couple relaxing picnic crane shot
                video                                            caption
7995  video10004.webm  astronaut-in-outer-space-against-the-backdrop-...
7996  video10005.webm  young-women-lying-on-sunbed-and-applying-sun-c...
7997  video10006.webm  doctor-talking-to-patient-using-a-tablet-to-ex...
7998  video10007.webm  businessman-sitting-on-the-beach-on-inflatable...
7999  video10008.webm  woman-eating-ice-cream-and-sitting-in-the-stre...


**Removing Stopwords from Captions**

In [64]:
!pip install PyPrind
from string import punctuation
import pyprind
from collections import Counter
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
#stopwords = nltk.corpus.stopwords.words('english')
stopwords = stopwords.words('english')
print(f'Length of Stopwords: {len(stopwords)}')

# setup prograss tracker
pbar = pyprind.ProgBar(len(Caption_dataset['caption']), title='Counting word occurrences')
for i, cap in enumerate(Caption_dataset['caption']):
    # replace punctuations with space
    # convert words to lower case 
    text = ''.join([c if c not in punctuation else ' ' for c in cap]).lower()
    #removing stopwords
    rmv_stopwords= ' '.join([word for word in text.split() if word not in stopwords])
    Caption_dataset.loc[i,'caption'] = rmv_stopwords #updating the original captions 
    pbar.update()

print("")
print(Caption_dataset.head())
print(Caption_dataset.shape)

Counting word occurrences


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Length of Stopwords: 179


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01



          video                                    caption
0   video3.webm                 blonde woman massaged tilt
1   video4.webm  roulette table spinning ball closeup shot
2   video6.webm                              khr gangsters
3   video8.webm          medical helicopter hovers airport
4  video10.webm          couple relaxing picnic crane shot
(8000, 2)


**Converting Captions into vector**

In [65]:
from sklearn.feature_extraction.text import CountVectorizer
cv_test = CountVectorizer()
count_vector_test=cv_test.fit_transform(Caption_dataset['caption'])
print(cv_test)

Caption_array_dataset=count_vector_test.toarray()
print(Caption_array_dataset.shape)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)
(8000, 5762)


**Merging Caption and C3D Features**

In [66]:
Caption_C3D_Test = np.concatenate((C3D_dataset, Caption_array_dataset), axis=1)
print(Caption_C3D_Test)
print(Caption_C3D_Test.shape)

[[1.085767e-02 1.038633e-02 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [1.995100e-04 6.494000e-05 9.938069e-01 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [0.000000e+00 0.000000e+00 0.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 ...
 [2.111000e-04 5.815590e-03 2.334000e-04 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [2.144980e-03 3.588087e-02 2.395500e-04 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [1.027290e-03 8.806000e-05 1.053500e-04 ... 0.000000e+00 0.000000e+00
  0.000000e+00]]
(8000, 5863)


### **Seperating Developement and Test Dataset**

In [67]:
X_train = Caption_C3D_Test[0:6000]
X_test = Caption_C3D_Test[6000:]
y_train = ground_truth[['short-term_memorability','long-term_memorability']].values

print('X_train', X_train.shape)
print('X_test', X_test.shape)
print('y_train', y_train.shape)

X_train (6000, 5863)
X_test (2000, 5863)
y_train (6000, 2)


### **Implementing and testing Models on C3D and Captions test dataset**

**SVR Model on Captions + C3d Test Dataset**

In [0]:
svr = SVR()
model = MultiOutputRegressor(estimator=svr)
model.fit(X_train,y_train)
#print(model.score(X_test,y_test))
predictions_cap_c3d_svr_test = model.predict(X_test)
#Get_score(predictions_cap_c3d_svr_test, y_test)

**Random Forest Model on Captions + C3d Test Dataset**

In [0]:
rf = RandomForestRegressor()
model = MultiOutputRegressor(estimator=rf)
model.fit(X_train,y_train)
#print(model.score(X_test,y_test))
predictions_cap_c3d_rf_test = model.predict(X_test)
#Get_score(predictions_cap_c3d_rf_test, y_test)

**LASSO Model on Captions + C3d Test Dataset**

In [0]:
from sklearn.linear_model import Lasso
ls = Lasso(alpha=1e-5,normalize=True, max_iter=1e5)
model = MultiOutputRegressor(estimator=ls)
model.fit(X_train,y_train)
#print(model.score(X_test,Y_test))
predictions_cap_c3d_lasso_test = model.predict(X_test)
#Get_score(predictions_cap_c3d_lasso_test, y_test)

**Ensambling results of LASSO and SVR for Captions + C3D on Test Dataset**

In [73]:
predictions_weighted_cap_c3d_test_LS  = (1/2)*predictions_cap_c3d_lasso_test + (1/2)*predictions_cap_c3d_svr_test
print(predictions_weighted_cap_c3d_test_LS)

[[0.83252636 0.79077252]
 [0.79278599 0.78819937]
 [0.8918482  0.86130627]
 ...
 [0.87294758 0.78954006]
 [0.89381662 0.84859123]
 [0.88449117 0.79860371]]


**Ensambling results of LASSO and RF for Captions + C3D on Test Dataset**

In [74]:
predictions_weighted_cap_c3d_test_LR  = (1/2)*predictions_cap_c3d_lasso_test + (1/2)*predictions_cap_c3d_rf_test
print(predictions_weighted_cap_c3d_test_LR)

[[0.8527453  0.78495859]
 [0.7813946  0.79940931]
 [0.91108879 0.83389429]
 ...
 [0.87431821 0.79051885]
 [0.90659443 0.82111307]
 [0.89078685 0.81878415]]


**Ensambling results of SRF and RF for Captions + C3D on Test Dataset**

In [75]:
predictions_weighted_cap_c3d_test_SR  = (1/2)*predictions_cap_c3d_svr_test + (1/2)*predictions_cap_c3d_rf_test
print(predictions_weighted_cap_c3d_test_SR)

[[0.84019106 0.77445393]
 [0.8090114  0.80486007]
 [0.87672941 0.78820198]
 ...
 [0.87596936 0.75914121]
 [0.87457219 0.79176816]
 [0.87369432 0.82155956]]


**Ensambling results of LASSO,SRF and RF for Captions + C3D on Test Dataset**

In [77]:
predictions_weighted_cap_c3d_test_all  = (1/3)*predictions_cap_c3d_svr_test + (1/3)*predictions_cap_c3d_rf_test + (1/3)*predictions_cap_c3d_lasso_test
print(predictions_weighted_cap_c3d_test_all)

[[0.84182091 0.78339502]
 [0.79439733 0.79748958]
 [0.89322214 0.82780085]
 ...
 [0.87441172 0.77973337]
 [0.89166108 0.82049082]
 [0.88299078 0.81298248]]


**Ensambled results with LASSO**

In [78]:
Get_score(predictions_cap_c3d_lasso_test, predictions_weighted_cap_c3d_test_all)

The Spearman's correlation coefficient is: 0.907
The Spearman's correlation coefficient is: 0.898


**Ensambled results with SVR**

In [79]:
Get_score(predictions_cap_c3d_svr_test, predictions_weighted_cap_c3d_test_all)

The Spearman's correlation coefficient is: 0.880
The Spearman's correlation coefficient is: 0.858


**Ensambled results with RF**

In [80]:
Get_score(predictions_cap_c3d_rf_test, predictions_weighted_cap_c3d_test_all)

The Spearman's correlation coefficient is: 0.824
The Spearman's correlation coefficient is: 0.568


**Exporting Results to .csv file**

In [0]:
df_result = pd.DataFrame(predictions_weighted_cap_c3d_test_all, columns=['short-term_memorability','long-term_memorability'])
df_result.to_csv('/content/drive/My Drive/Machine Learning/Apurva_Saswadkar_19210067_Predictions.csv',index=None)

In [0]:
Predictions_Cap_C3D_Test = pd.read_csv('/content/drive/My Drive/Machine Learning/Predictions_Cap_C3D.csv',header=None)
Predictions_Cap_C3D_Test.head()

,0,1
0,0.860510,0.803184
1,0.847700,0.778529
2,0.842546,0.708117
3,0.876340,0.881570
4,0.798642,0.595775
